# ML From Scratch - 머신러닝 알고리즘 직접 구현

## 개요
이 노트북은 주요 머신러닝 알고리즘을 **라이브러리 없이 직접 구현**하고,
sklearn 라이브러리와 성능을 비교합니다.

### 구현된 알고리즘
1. **Decision Tree Regressor** - CART 기반 결정 트리
2. **Gradient Boosting Regressor** - 잔차 학습 기반 부스팅
3. **AdaBoost Regressor** - 가중치 기반 부스팅 (AdaBoost.R2)
4. **XGBoost Regressor** - 2차 근사 + 정규화 기반 부스팅
5. **Random Forest Regressor** - 배깅 + 랜덤 피처 선택

### 핵심 차별점
- NumPy만 사용하여 알고리즘의 **수학적 원리**를 명확히 구현
- 학습 과정을 **시각화**하여 알고리즘 동작 원리 이해
- sklearn과 **동일한 인터페이스**로 쉬운 비교 가능

---
## 1. 환경 설정 및 모듈 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# From Scratch 구현 임포트
import sys
sys.path.insert(0, '.')

from ml_from_scratch import (
    DecisionTreeRegressor as DTScratch,
    GradientBoostingRegressor as GBScratch,
    AdaBoostRegressor as AdaScratch,
    XGBoostRegressor as XGBScratch,
    RandomForestRegressor as RFScratch,
    MLVisualizer
)

# sklearn 비교용
from sklearn.tree import DecisionTreeRegressor as DTSklearn
from sklearn.ensemble import (
    GradientBoostingRegressor as GBSklearn,
    AdaBoostRegressor as AdaSklearn,
    RandomForestRegressor as RFSklearn
)
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# 시각화 도구 초기화
viz = MLVisualizer()

print("모듈 임포트 완료!")
print(f"From Scratch 모듈 경로: {sys.path[0]}")

---
## 2. 합성 데이터로 알고리즘 검증

sklearn과 동일한 결과가 나오는지 확인하기 위해 간단한 합성 데이터로 테스트합니다.

In [ ]:
# 합성 데이터 생성
np.random.seed(42)

n_samples = 500
n_features = 10

X_synth = np.random.randn(n_samples, n_features)

# 비선형 관계: y = 2*x0 + x1^2 - 0.5*x2*x3 + noise
y_synth = (
    2 * X_synth[:, 0] + 
    X_synth[:, 1] ** 2 - 
    0.5 * X_synth[:, 2] * X_synth[:, 3] +
    np.random.randn(n_samples) * 0.5
)

# Train/Test 분할 (시간 순서 가정)
split_idx = int(n_samples * 0.8)
X_train_s, X_test_s = X_synth[:split_idx], X_synth[split_idx:]
y_train_s, y_test_s = y_synth[:split_idx], y_synth[split_idx:]

print(f"합성 데이터 생성 완료")
print(f"Train: {X_train_s.shape}, Test: {X_test_s.shape}")
print(f"y 범위: [{y_synth.min():.2f}, {y_synth.max():.2f}]")

### 2.1 Decision Tree 비교

In [ ]:
print("="*60)
print("Decision Tree Regressor 비교")
print("="*60)

# From Scratch
dt_scratch = DTScratch(max_depth=5, min_samples_split=5, random_state=42)
dt_scratch.fit(X_train_s, y_train_s)
pred_scratch = dt_scratch.predict(X_test_s)
rmse_scratch = np.sqrt(mean_squared_error(y_test_s, pred_scratch))

# sklearn
dt_sklearn = DTSklearn(max_depth=5, min_samples_split=5, random_state=42)
dt_sklearn.fit(X_train_s, y_train_s)
pred_sklearn = dt_sklearn.predict(X_test_s)
rmse_sklearn = np.sqrt(mean_squared_error(y_test_s, pred_sklearn))

print(f"\n[From Scratch]")
print(f"  RMSE: {rmse_scratch:.4f}")
print(f"  트리 깊이: {dt_scratch.get_depth()}")
print(f"  리프 수: {dt_scratch.get_n_leaves()}")

print(f"\n[sklearn]")
print(f"  RMSE: {rmse_sklearn:.4f}")
print(f"  트리 깊이: {dt_sklearn.get_depth()}")
print(f"  리프 수: {dt_sklearn.get_n_leaves()}")

print(f"\n[성능 차이]")
print(f"  RMSE 차이: {abs(rmse_scratch - rmse_sklearn):.6f}")
print(f"  예측값 상관계수: {np.corrcoef(pred_scratch, pred_sklearn)[0,1]:.6f}")

In [ ]:
# Decision Tree 구조 시각화
feature_names = [f'X{i}' for i in range(n_features)]
fig = viz.plot_decision_tree(
    dt_scratch, 
    feature_names=feature_names,
    max_depth=3,
    title="Decision Tree Structure (From Scratch)"
)
plt.show()

### 2.2 Gradient Boosting 비교

In [ ]:
print("="*60)
print("Gradient Boosting Regressor 비교")
print("="*60)

# 공통 파라미터
gb_params = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 3,
    'random_state': 42
}

# From Scratch
gb_scratch = GBScratch(**gb_params, verbose=0)
gb_scratch.fit(X_train_s, y_train_s)
pred_scratch = gb_scratch.predict(X_test_s)
rmse_scratch = np.sqrt(mean_squared_error(y_test_s, pred_scratch))

# sklearn
gb_sklearn = GBSklearn(**gb_params)
gb_sklearn.fit(X_train_s, y_train_s)
pred_sklearn = gb_sklearn.predict(X_test_s)
rmse_sklearn = np.sqrt(mean_squared_error(y_test_s, pred_sklearn))

print(f"\n[From Scratch]")
print(f"  RMSE: {rmse_scratch:.4f}")
print(f"  학습된 트리 수: {len(gb_scratch.estimators_)}")

print(f"\n[sklearn]")
print(f"  RMSE: {rmse_sklearn:.4f}")
print(f"  학습된 트리 수: {len(gb_sklearn.estimators_)}")

print(f"\n[성능 차이]")
print(f"  RMSE 차이: {abs(rmse_scratch - rmse_sklearn):.4f}")
print(f"  상대 오차: {abs(rmse_scratch - rmse_sklearn) / rmse_sklearn * 100:.2f}%")

In [ ]:
# Gradient Boosting 학습 곡선
fig = viz.plot_boosting_curve(
    gb_scratch,
    title="Gradient Boosting Learning Curve (From Scratch)"
)
plt.show()

### 2.3 AdaBoost 비교

In [ ]:
print("="*60)
print("AdaBoost Regressor 비교")
print("="*60)

# 공통 파라미터
ada_params = {
    'n_estimators': 50,
    'learning_rate': 1.0,
    'random_state': 42
}

# From Scratch (max_depth 추가)
ada_scratch = AdaScratch(**ada_params, max_depth=3, loss='linear')
ada_scratch.fit(X_train_s, y_train_s)
pred_scratch = ada_scratch.predict(X_test_s)
rmse_scratch = np.sqrt(mean_squared_error(y_test_s, pred_scratch))

# sklearn (base_estimator 지정)
from sklearn.tree import DecisionTreeRegressor as DTBase
ada_sklearn = AdaSklearn(
    estimator=DTBase(max_depth=3),
    **ada_params,
    loss='linear'
)
ada_sklearn.fit(X_train_s, y_train_s)
pred_sklearn = ada_sklearn.predict(X_test_s)
rmse_sklearn = np.sqrt(mean_squared_error(y_test_s, pred_sklearn))

print(f"\n[From Scratch]")
print(f"  RMSE: {rmse_scratch:.4f}")
print(f"  학습된 트리 수: {len(ada_scratch.estimators_)}")

print(f"\n[sklearn]")
print(f"  RMSE: {rmse_sklearn:.4f}")
print(f"  학습된 트리 수: {len(ada_sklearn.estimators_)}")

print(f"\n[성능 차이]")
print(f"  RMSE 차이: {abs(rmse_scratch - rmse_sklearn):.4f}")

In [ ]:
# AdaBoost 학습 곡선
fig = viz.plot_boosting_curve(
    ada_scratch,
    title="AdaBoost Learning Curve (From Scratch)"
)
plt.show()

### 2.4 XGBoost 비교

In [ ]:
print("="*60)
print("XGBoost Regressor 비교")
print("="*60)

# From Scratch XGBoost
xgb_scratch = XGBScratch(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    reg_lambda=1.0,
    reg_gamma=0.0,
    random_state=42,
    verbose=0
)
xgb_scratch.fit(X_train_s, y_train_s)
pred_scratch = xgb_scratch.predict(X_test_s)
rmse_scratch = np.sqrt(mean_squared_error(y_test_s, pred_scratch))

print(f"\n[XGBoost From Scratch]")
print(f"  RMSE: {rmse_scratch:.4f}")
print(f"  학습된 트리 수: {len(xgb_scratch.estimators_)}")

# xgboost 라이브러리와 비교 (설치된 경우)
try:
    import xgboost as xgb
    xgb_lib = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=4,
        reg_lambda=1.0,
        reg_alpha=0.0,
        random_state=42
    )
    xgb_lib.fit(X_train_s, y_train_s)
    pred_lib = xgb_lib.predict(X_test_s)
    rmse_lib = np.sqrt(mean_squared_error(y_test_s, pred_lib))
    
    print(f"\n[XGBoost Library]")
    print(f"  RMSE: {rmse_lib:.4f}")
    print(f"\n[성능 차이]")
    print(f"  RMSE 차이: {abs(rmse_scratch - rmse_lib):.4f}")
except ImportError:
    print("\nxgboost 라이브러리가 설치되지 않아 비교를 건너뜁니다.")

In [ ]:
# XGBoost 학습 곡선
fig = viz.plot_boosting_curve(
    xgb_scratch,
    title="XGBoost Learning Curve (From Scratch)"
)
plt.show()

### 2.5 Random Forest 비교

In [ ]:
print("="*60)
print("Random Forest Regressor 비교")
print("="*60)

# 공통 파라미터
rf_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 5,
    'random_state': 42
}

# From Scratch
rf_scratch = RFScratch(**rf_params, max_features='sqrt', oob_score=True, verbose=0)
rf_scratch.fit(X_train_s, y_train_s)
pred_scratch = rf_scratch.predict(X_test_s)
rmse_scratch = np.sqrt(mean_squared_error(y_test_s, pred_scratch))

# sklearn
rf_sklearn = RFSklearn(**rf_params, max_features='sqrt', oob_score=True)
rf_sklearn.fit(X_train_s, y_train_s)
pred_sklearn = rf_sklearn.predict(X_test_s)
rmse_sklearn = np.sqrt(mean_squared_error(y_test_s, pred_sklearn))

print(f"\n[From Scratch]")
print(f"  RMSE: {rmse_scratch:.4f}")
print(f"  OOB Score: {rf_scratch.oob_score_:.4f}" if rf_scratch.oob_score_ else "  OOB Score: N/A")

print(f"\n[sklearn]")
print(f"  RMSE: {rmse_sklearn:.4f}")
print(f"  OOB Score: {rf_sklearn.oob_score_:.4f}")

print(f"\n[성능 차이]")
print(f"  RMSE 차이: {abs(rmse_scratch - rmse_sklearn):.4f}")

In [ ]:
# Random Forest 수렴 과정
fig = viz.plot_ensemble_convergence(
    rf_scratch,
    X_test_s,
    y_test_s,
    title="Random Forest Prediction Convergence (From Scratch)"
)
plt.show()

---
## 3. 피처 중요도 비교

In [ ]:
# 모든 모델의 피처 중요도 비교
models_for_importance = {
    'Decision Tree': dt_scratch,
    'Gradient Boosting': gb_scratch,
    'XGBoost': xgb_scratch,
    'Random Forest': rf_scratch
}

fig = viz.plot_feature_importance(
    models_for_importance,
    feature_names=feature_names,
    top_k=10,
    title="Feature Importance Comparison (From Scratch Models)"
)
plt.show()

---
## 4. 전체 모델 성능 요약

In [ ]:
# 성능 요약 테이블
def evaluate_model(model, X_test, y_test):
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    mae = mean_absolute_error(y_test, pred)
    mape = np.mean(np.abs((y_test - pred) / y_test)) * 100
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

results_scratch = {
    'Decision Tree': evaluate_model(dt_scratch, X_test_s, y_test_s),
    'Gradient Boosting': evaluate_model(gb_scratch, X_test_s, y_test_s),
    'AdaBoost': evaluate_model(ada_scratch, X_test_s, y_test_s),
    'XGBoost': evaluate_model(xgb_scratch, X_test_s, y_test_s),
    'Random Forest': evaluate_model(rf_scratch, X_test_s, y_test_s)
}

results_sklearn = {
    'Decision Tree': evaluate_model(dt_sklearn, X_test_s, y_test_s),
    'Gradient Boosting': evaluate_model(gb_sklearn, X_test_s, y_test_s),
    'AdaBoost': evaluate_model(ada_sklearn, X_test_s, y_test_s),
    'Random Forest': evaluate_model(rf_sklearn, X_test_s, y_test_s)
}

# 데이터프레임으로 정리
df_scratch = pd.DataFrame(results_scratch).T
df_sklearn = pd.DataFrame(results_sklearn).T

print("="*60)
print("From Scratch 모델 성능")
print("="*60)
print(df_scratch.round(4).to_string())

print("\n" + "="*60)
print("sklearn 모델 성능")
print("="*60)
print(df_sklearn.round(4).to_string())

In [ ]:
# 성능 비교 시각화
fig = viz.plot_model_comparison(
    results_scratch,
    metrics=['RMSE', 'MAE'],
    title="From Scratch Model Performance (Synthetic Data)"
)
plt.show()

---
## 5. 니켈 가격 데이터에 적용

실제 니켈 가격 데이터에 From Scratch 모델을 적용합니다.

In [ ]:
# 데이터 로드
df_raw = pd.read_csv('data_weekly_260120.csv')
df_raw['dt'] = pd.to_datetime(df_raw['dt'])
df_raw = df_raw.set_index('dt').sort_index()

print(f"원본 데이터: {df_raw.shape}")
print(f"기간: {df_raw.index.min()} ~ {df_raw.index.max()}")

In [ ]:
# 피처 선택 (LME Index 제외)
target_col = 'Com_LME_Ni_Cash'

# 필터링: 타겟 + 지수(Idx_) + 채권(Bonds_) + 환율(EX_) + 금속(Com_LME*)
def filter_cols(cols):
    keep = [target_col]
    for c in cols:
        if c == target_col:
            continue
        if c.startswith('Idx_') or c.startswith('Bonds_') or c.startswith('EX_'):
            keep.append(c)
        elif c.startswith('Com_LME') and 'Index' not in c:  # LME Index 제외
            keep.append(c)
        elif c.startswith('Com_') and 'LME' not in c:
            keep.append(c)
    return keep

filtered_cols = filter_cols(df_raw.columns)
df = df_raw[filtered_cols].copy()
df = df.ffill().bfill()

print(f"필터링 후: {df.shape}")
print(f"피처 수: {len(filtered_cols) - 1}")

In [ ]:
# 타겟과 피처 분리 (1주 지연 적용 - 데이터 누수 방지)
y = df[target_col].copy()
X = df.drop(columns=[target_col]).shift(1)  # t-1 시점 정보만 사용

# NaN 제거
valid_idx = X.dropna().index.intersection(y.dropna().index)
X = X.loc[valid_idx]
y = y.loc[valid_idx]

print(f"최종 데이터: X={X.shape}, y={y.shape}")
print(f"\n주의: shift(1) 적용으로 t-1 시점 정보만 사용 (데이터 누수 방지)")

In [ ]:
# Train/Val/Test 분할 (시간 기반)
CONFIG = {
    'val_start': '2025-08-04',
    'val_end': '2025-10-20',
    'test_start': '2025-10-27',
    'test_end': '2026-01-12'
}

train_mask = X.index < CONFIG['val_start']
val_mask = (X.index >= CONFIG['val_start']) & (X.index <= CONFIG['val_end'])
test_mask = (X.index >= CONFIG['test_start']) & (X.index <= CONFIG['test_end'])

X_train = X[train_mask].values
y_train = y[train_mask].values
X_val = X[val_mask].values
y_val = y[val_mask].values
X_test = X[test_mask].values
y_test = y[test_mask].values

print(f"Train: {X_train.shape[0]} samples ({X.index[train_mask].min()} ~ {X.index[train_mask].max()})")
print(f"Val: {X_val.shape[0]} samples ({CONFIG['val_start']} ~ {CONFIG['val_end']})")
print(f"Test: {X_test.shape[0]} samples ({CONFIG['test_start']} ~ {CONFIG['test_end']})")

In [ ]:
# From Scratch 모델 학습 및 평가
print("="*60)
print("From Scratch 모델 학습 (니켈 가격 예측)")
print("="*60)

nickel_results = {}

# 1. Gradient Boosting
print("\n[1] Gradient Boosting 학습 중...")
gb_nickel = GBScratch(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    random_state=42,
    verbose=0
)
gb_nickel.fit(X_train, y_train, X_val, y_val)
pred_gb = gb_nickel.predict(X_test)
nickel_results['GB_Scratch'] = {
    'RMSE': np.sqrt(mean_squared_error(y_test, pred_gb)),
    'MAE': mean_absolute_error(y_test, pred_gb),
    'predictions': pred_gb
}
print(f"  Test RMSE: {nickel_results['GB_Scratch']['RMSE']:.2f}")

# 2. XGBoost
print("\n[2] XGBoost 학습 중...")
xgb_nickel = XGBScratch(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    reg_lambda=1.0,
    subsample=0.8,
    random_state=42,
    verbose=0
)
xgb_nickel.fit(X_train, y_train, X_val, y_val)
pred_xgb = xgb_nickel.predict(X_test)
nickel_results['XGB_Scratch'] = {
    'RMSE': np.sqrt(mean_squared_error(y_test, pred_xgb)),
    'MAE': mean_absolute_error(y_test, pred_xgb),
    'predictions': pred_xgb
}
print(f"  Test RMSE: {nickel_results['XGB_Scratch']['RMSE']:.2f}")

# 3. Random Forest
print("\n[3] Random Forest 학습 중...")
rf_nickel = RFScratch(
    n_estimators=200,
    max_depth=10,
    max_features='sqrt',
    random_state=42,
    verbose=0
)
rf_nickel.fit(X_train, y_train)
pred_rf = rf_nickel.predict(X_test)
nickel_results['RF_Scratch'] = {
    'RMSE': np.sqrt(mean_squared_error(y_test, pred_rf)),
    'MAE': mean_absolute_error(y_test, pred_rf),
    'predictions': pred_rf
}
print(f"  Test RMSE: {nickel_results['RF_Scratch']['RMSE']:.2f}")

# 4. AdaBoost
print("\n[4] AdaBoost 학습 중...")
ada_nickel = AdaScratch(
    n_estimators=100,
    learning_rate=0.5,
    max_depth=3,
    random_state=42
)
ada_nickel.fit(X_train, y_train)
pred_ada = ada_nickel.predict(X_test)
nickel_results['Ada_Scratch'] = {
    'RMSE': np.sqrt(mean_squared_error(y_test, pred_ada)),
    'MAE': mean_absolute_error(y_test, pred_ada),
    'predictions': pred_ada
}
print(f"  Test RMSE: {nickel_results['Ada_Scratch']['RMSE']:.2f}")

In [ ]:
# Naive 베이스라인 추가
print("\n[5] Naive 베이스라인 계산 중...")
y_series = y[test_mask]
prev_price = y.shift(1)[test_mask].values
prev_prev_price = y.shift(2)[test_mask].values

# Naive Drift: prev + (prev - prev_prev)
naive_drift = prev_price + (prev_price - prev_prev_price)
nickel_results['Naive_Drift'] = {
    'RMSE': np.sqrt(mean_squared_error(y_test, naive_drift)),
    'MAE': mean_absolute_error(y_test, naive_drift),
    'predictions': naive_drift
}
print(f"  Test RMSE: {nickel_results['Naive_Drift']['RMSE']:.2f}")

In [ ]:
# 결과 요약
print("\n" + "="*60)
print("니켈 가격 예측 결과 요약 (Test Period)")
print("="*60)

summary_df = pd.DataFrame({
    name: {'RMSE': r['RMSE'], 'MAE': r['MAE']}
    for name, r in nickel_results.items()
}).T.sort_values('RMSE')

print(summary_df.round(2).to_string())

In [ ]:
# 예측 결과 시각화
test_dates = X[test_mask].index

predictions_dict = {
    name: r['predictions'] for name, r in nickel_results.items()
}

fig = viz.plot_prediction_timeline(
    test_dates,
    y_test,
    predictions_dict,
    title="Nickel Price Prediction - Test Period (From Scratch Models)"
)
plt.show()

In [ ]:
# 잔차 분석 (최고 성능 모델)
best_model_name = summary_df.index[0]
best_pred = nickel_results[best_model_name]['predictions']

fig = viz.plot_residual_analysis(
    y_test,
    best_pred,
    title=f"Residual Analysis - {best_model_name}"
)
plt.show()

In [ ]:
# Gradient Boosting 학습 곡선 (니켈 데이터)
fig = viz.plot_boosting_curve(
    gb_nickel,
    title="Gradient Boosting Learning Curve (Nickel Data)",
    show_validation=True
)
plt.show()

In [ ]:
# 피처 중요도 (니켈 데이터)
feature_names_nickel = X.columns.tolist()

nickel_models = {
    'GB_Scratch': gb_nickel,
    'XGB_Scratch': xgb_nickel,
    'RF_Scratch': rf_nickel
}

fig = viz.plot_feature_importance(
    nickel_models,
    feature_names=feature_names_nickel,
    top_k=15,
    title="Feature Importance - Nickel Price Prediction (From Scratch)"
)
plt.show()

---
## 6. 알고리즘 수학적 원리 요약

### 6.1 Decision Tree
- **분할 기준**: MSE 감소 최대화
- **Gain** = MSE_parent - MSE_split
- **예측**: 리프 노드에 속한 샘플들의 평균

### 6.2 Gradient Boosting
- **핵심**: 잔차(residual)를 순차적으로 학습
- **업데이트**: F_m(x) = F_{m-1}(x) + η * h_m(x)
- **잔차**: r = y - F_{m-1}(x) (음의 그래디언트)

### 6.3 AdaBoost.R2
- **핵심**: 어려운 샘플에 높은 가중치
- **손실**: L_i = |y_i - h_m(x_i)| / D (정규화된 오차)
- **가중치 업데이트**: w_i = w_i * β^(1-L_i)

### 6.4 XGBoost
- **2차 테일러 전개**: L ≈ g*Δ + (1/2)*h*Δ²
- **최적 리프 가중치**: w* = -G / (H + λ)
- **분할 이득**: Gain = ½[G_L²/(H_L+λ) + G_R²/(H_R+λ) - G²/(H+λ)] - γ

### 6.5 Random Forest
- **배깅**: 복원 추출로 다양한 트리 학습
- **랜덤 피처**: 각 분할에서 sqrt(n_features) 피처만 고려
- **분산 감소**: Var(평균) ≈ Var(개별) / n

---
## 7. 결론

### 검증 결과
1. **From Scratch 구현**이 sklearn과 유사한 성능을 보임
2. **XGBoost의 2차 근사**가 일반 Gradient Boosting보다 효율적
3. **Random Forest의 OOB 점수**가 검증 세트 없이도 일반화 성능 추정 가능

### 니켈 가격 예측
- ML 모델들이 Naive 베이스라인보다 성능이 낮음
- 이는 Test 기간의 **일방적 추세** 때문
- Hybrid 모델(Naive + ML)이 가장 효과적일 수 있음

### 핵심 학습 포인트
1. 알고리즘의 **수학적 원리**를 직접 구현하며 이해
2. **학습 과정 시각화**를 통한 동작 원리 파악
3. **데이터 누수 방지**의 중요성 (shift(1) 적용)